In [1]:
from main.fd_violation import fd_violations
import pandas as pd

# 簡単なデータ例

In [2]:
df = pd.DataFrame({'a':['A','A','C','D'],
                    'b':['AA','BB','CC','DD']})
df = df.astype({'a': 'string', 'b': 'string'})
groups = fd_violations(df, lhs='a', rhs='b')

In [3]:
groups

{'A': Counter({'AA': 1, 'BB': 1})}

In [4]:
df = pd.DataFrame({'a':['A','A','A','D'],
                    'b':['AA','BB','BB','DD']})
df = df.astype({'a': 'string', 'b': 'string'})
groups = fd_violations(df, lhs='a', rhs='b')

In [5]:
groups

{'A': Counter({'AA': 1, 'BB': 2})}

# opencleanデータによる現実的な例

元データ: https://github.com/VIDA-NYU/openclean/tree/master/examples/notebooks/data

In [6]:
datafile = 'data/jt7v-77mi.tsv'
df = pd.read_table(datafile)

In [7]:
df.head(5)

,Summons Number,Plate ID,Registration State,Plate Type,Issue Date,Violation Code,Vehicle Body Type,Vehicle Make,Issuing Agency,Street Code1,...,Vehicle Color,Unregistered Vehicle?,Vehicle Year,Meter Number,Feet From Curb,Violation Post Code,Violation Description,No Standing or Stopping Violation,Hydrant Violation,Double Parking Violation
0,1361929741,FCJ5493,NY,PAS,12/18/1970,20,SUBN,GMC,S,35030,...,BLACK,0.0,2013,-,0,NaN,NaN,NaN,NaN,NaN
1,1366962000,63540MC,NY,COM,02/02/1971,46,DELV,FRUEH,P,58830,...,BRN,0.0,2013,-,0,NaN,NaN,NaN,NaN,NaN
2,1342296187,GCY4187,NY,SRF,09/18/1971,21,VAN,FORD,S,11790,...,BLUE,0.0,2002,-,0,NaN,NaN,NaN,NaN,NaN
3,1342296199,95V6675,TX,PAS,09/18/1971,21,NaN,GMC,S,11790,...,SILVR,0.0,2008,-,0,NaN,NaN,NaN,NaN,NaN
4,1342296217,FYM5117,NY,SRF,09/18/1971,21,SUBN,NISSA,S,28190,...,WHITE,0.0,2012,-,0,NaN,NaN,NaN,NaN,NaN


In [8]:
# 利用列を定義
columns = [
    'Plate ID',
    'Registration State',
    'Plate Type',
    'Meter Number',
    'Street',
    'Vehicle Body Type',
    'Vehicle Make',
    'Vehicle Color'
]

In [11]:
# 入力データの整形
df = df[columns]
df = df.astype('string')
df = df[df['Meter Number'] != '-']
df = df[df['Meter Number'].notna()]

In [12]:
groups = fd_violations(df, lhs='Meter Number', rhs='Street')

In [13]:
# 検出結果件数の表示
for key in groups:
    print('{} {}'.format(key, len(groups.get(key))))

144-3937 2
143-3785 2
144-6376 2


In [17]:
# 検出結果明細の表示
print('Meter Number | Street (Count)')
print('=============|===============')
for key in groups:
    conflicts = groups.get(key).most_common()
    street, count = conflicts[0]
    print('{:<12s} | {} x {}'.format(key, count, street))
    for street, count in conflicts[1:]:
        print('             | {} x {}'.format(count, street))
    print('-------------|---------------')

Meter Number | Street (Count)
=============|===============
144-3937     | 1 x WEST 42 STREET
             | 1 x WEST 42 ST
-------------|---------------
143-3785     | 2 x WEST 43RD ST
             | 1 x WEST 43 ST
-------------|---------------
144-6376     | 1 x 8TH AVE
             | 1 x 8TH AVENUE
-------------|---------------
